# RAC step 2:
1. select data
2. fit data to a Pade approximant
3. analytical continuation to $\lambda = 0$

see *J. Chem. Phys.* **143**, 184102 (2015)

In [1]:
show_plot=False

In [2]:
import sys
import numpy as np
from scipy.optimize import least_squares
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib qt5
#import matplotlib
#matplotlib.use('Qt5Agg')

library with Pade approximants and their second derivatives 

In [3]:
sys.path.append('../../RAC_LIBS')
from rac_aux import *
from params import Params
from fitman import bh_with_lsq
au2eV=27.2114

### Select Pade order ([3,1] is recommended.)

In [4]:
nm=31

pade_fns = {"21":(pade_21_lsq, pade_21j_lsq), 
            "31":(pade_31_lsq, pade_31j_lsq), 
            "32":(pade_32_lsq, pade_32j_lsq),
            "41":(pade_41_lsq, pade_41j_lsq),
            "42":(pade_42_lsq, pade_42j_lsq),
            "43":(pade_43_lsq, pade_43j_lsq),
            "52":(pade_52_lsq, pade_52j_lsq),
            "53":(pade_53_lsq, pade_53j_lsq)}

fun=pade_fns[str(nm)][0]
jac=pade_fns[str(nm)][1]

Set data either to `total` or to `relative`. The former is for CCSD(T) total energies for neutral and anion; the latter is for EOM-CCSD attachment data. For the Jolanta potential, all energies are `relative`.

Also, for standard Jolanta-3D, the resonance is the second state. 

In [55]:
#data = 'total'
data = 'relative'
n_state = 'E2'
if data == 'total':
    df = pd.read_csv('CCSDpT_data.csv', delim_whitespace=True)
    df['Eb'] = (df['E1'] - df['E0'])*au2eV
elif data == 'relative':
    #df = pd.read_csv('rac_DVR_b-scale_rmax_12.csv')
    #df = pd.read_csv('rac_DVR_coulomb_rmax_12.csv')
    #df = pd.read_csv('rac_DVR_softbox_rmax_12.csv')
    #df = pd.read_csv('rac_GTO_TZ_0_b-scale.csv')
    #df = pd.read_csv('rac_GTO_TZ_0_coulomb.csv')
    df = pd.read_csv('rac_GTO_TZ_0_softbox.csv')
    df['Eb']=df[n_state]
else:
    print('Data organization not yet implemented:', data)
df.head()

,z,E1,E2,Eb
0,0.000,-7.170439,3.298521,3.298521
1,0.013,-7.506676,3.057427,3.057427
2,0.026,-7.843046,2.814717,2.814717
3,0.039,-8.179546,2.570440,2.570440
4,0.052,-8.516174,2.324650,2.324650


## Data selection
Rule of thumb: use all $E_b < -0.5$ eV

In [56]:
Estart=-0.5
n_first=np.argmin(np.abs(df.Eb.values-Estart))
z0=df['z'][0]
ls=np.copy(df.z.values[n_first:]) - z0
Es=np.copy(df.E2.values[n_first:])
Nval=len(ls)
print('%d bound energies between %f and %f' % (Nval, Es[0], Es[-1]))
if show_plot:
    plt.cla()
    plt.plot(ls,Es)
    plt.show()

86 bound energies between -0.463485 and -23.659219


Prepare $\kappa^2=-E_b$, $\kappa$, and fitting weights (set to 1) which are all going to be passed to the optimizer.

In [57]:
k2s = -Es
ks  = np.sqrt(k2s)
sigmas = weights(Nval, 'ones')
parg=(ks,k2s,ls,sigmas)

## Fitting procedure
1. Heuristic guess on the basis of linear real extrapolation
2. Fit all selected binding energies to get good guess for $E_r$ 
3. Average over various selection ranges from -0.5 to $n \times E_r$, $n=4-7$

In [58]:
E0 = 0.6*linear_extra(ls,Es)
P0 = Params(E0, ls[0])
p0 = np.array(P0.start(str(nm), adhoc=True, noise=0.05))
print("Start parameters: ", p0)

Start parameters:  [0.19482518 0.27406077 1.52029792 0.82649047]


In [59]:
n_bh=200
sane_bnds = (E0-1, E0+1, 0.01, 1.5)
args = (ks, k2s, ls, sigmas, fun, jac)
best, sane, p0s, df = bh_with_lsq(p0, n_bh, args, sane_bnds, T=1e-4, verbose=1)

  Doing 200 basin hops.
  minimization failures: 0
  0 sane minima found
  Best:  3.461630  0.759058  1.3589e-08
  Sane:  0.000000  0.000000  0.0000e+00


In [60]:
df['Er_rounded']=np.round(df['Er'], decimals=4)
df['G_rounded']=np.round(df['G'], decimals=5)
df.drop_duplicates(subset=["Er_rounded", "G_rounded"])

,chis,Er,G,Er_rounded,G_rounded
0,1.358918e-08,3.461629,0.759057,3.4616,0.75906
1,1.358918e-08,3.461634,0.759065,3.4616,0.75907
2,1.358918e-08,3.461626,0.759053,3.4616,0.75905
167,1.358918e-08,3.461617,0.759038,3.4616,0.75904


Fit `Nval` data points to get a good guess for $E_r$ and provide abundant output to check the optimizer.

In [61]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(fun, p0s, method='trf', jac=jac, args=parg)
print("Jacobi-evaluations, njev:",res.njev)
print("chi**2 = 2*cost:",res.cost)
print("gradient:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
(Er,G) = res_ene(res.x[1], res.x[2])
print("Er=%f,  Gamma=%f" % (Er,G))

Least squares, trust-region-reflective (default) with analytic jac
Jacobi-evaluations, njev: 1
chi**2 = 2*cost: 6.794590451626926e-09
gradient: [-6.84645418e-11  1.30602448e-12  9.75577454e-12 -1.25851665e-12]
message: `gtol` termination condition is satisfied.
success: True
x: [0.17018621 0.319126   1.86333055 0.89821535]
chi2 = 1.359e-08
Er=3.461629,  Gamma=0.759057


### Energy ranges for averaging procedure
* Start with all energies from $-0.5$ to $-4*E_r$
* Stop with all energies from $-0.5$ to $-7*E_r$

In [62]:
Estop1=-4.0*Er
Estop2=-7.0*Er
jmx1=np.argmin(np.abs(Es-Estop1))
jmx2=np.argmin(np.abs(Es-Estop2))
print('First energy:                  E=%.5f' % (Es[0]))
print('Index for -4*Er0:         %3d; E=%.5f' % (jmx1,Es[jmx1]))
print('Index for -7*Er0:         %3d; E=%.5f' % (jmx2,Es[jmx2]))
print('Last index:               %3d; E=%.5f' % (Nval-1,Es[-1]))

First energy:                  E=-0.46349
Index for -4*Er0:          49; E=-13.71398
Index for -7*Er0:          85; E=-23.65922
Last index:                85; E=-23.65922


Loop over different ranges: 
* start with $\lambda(E=-0.5)$ to $\lambda(E=-4)$
* stop with $\lambda(E=-0.5)$ to $\lambda(E=-7)$

In [63]:
Emaxs = []
Ers = []
Gs = []
chi2s = []
print("  'lambda'       Emax/Er        Er       Gamma      chi**2")
for j in range(jmx1,jmx2+1):
    parg=(ks[:j], k2s[:j], ls[:j], sigmas[:j])
    res = least_squares(fun, p0s, method='trf', jac=jac, args=parg)
    chi2s.append(res.cost*2)
    Er, G = res_ene(res.x[1], res.x[2])
    print("%10f    %10f   %10f %10f   %.3e" % (ls[j], -Es[j]/Er, Er, G, res.cost*2))
    Emaxs.append(-Es[j]/Er)
    Ers.append(Er)
    Gs.append(G)

  'lambda'       Emax/Er        Er       Gamma      chi**2
  0.832000      4.089950     3.353093   0.465921   7.881e-10
  0.845000      4.170829     3.354114   0.475108   8.748e-10
  0.858000      4.251558     3.355233   0.484360   9.688e-10
  0.871000      4.332128     3.356450   0.493610   1.071e-09
  0.884000      4.412525     3.357768   0.502902   1.181e-09
  0.897000      4.492741     3.359188   0.512192   1.299e-09
  0.910000      4.572765     3.360713   0.521472   1.427e-09
  0.923000      4.652588     3.362344   0.530730   1.565e-09
  0.936000      4.732194     3.364085   0.539996   1.713e-09
  0.949000      4.811576     3.365937   0.549239   1.872e-09
  0.962000      4.890721     3.367901   0.558455   2.042e-09
  0.975000      4.969620     3.369980   0.567636   2.224e-09
  0.988000      5.048259     3.372176   0.576786   2.419e-09
  1.001000      5.126632     3.374488   0.585874   2.627e-09
  1.014000      5.204737     3.376912   0.594869   2.849e-09
  1.027000      5.282555  

Compute simple and weighted averages using $\chi^2$ as weights. 

In [64]:
Ers=np.array(Ers)
Gs=np.array(Gs)
chi2s=np.array(chi2s)
print("Simple average")
print("  Er = %f   STD = %f" % (np.mean(Ers), np.std(Ers)))
print("  G  = %f   STD = %f" % (np.mean(Gs), np.std(Gs)))
print("Weighted average:")
print("  Er = %f" % (np.average(Ers,weights=1.0/chi2s)))
print("  G  = %f" % (np.average(Gs,weights=1.0/chi2s)))

Simple average
  Er = 3.394063   STD = 0.031592
  G  = 0.623076   STD = 0.087952
Weighted average:
  Er = 3.372211
  G  = 0.554514
